In [12]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-nano", model_provider="openai")

In [13]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY")

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\PMYLS\\customer-support-chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PMYLS\\customer-support-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 94


In [14]:
#Open AI embeddings
def download_hugging_face_embeddings():
    embeddings=embeddings
    return embeddings


In [15]:
## Test the model
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 1536


## Set up pincone


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')


In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

cscpc = Pinecone(api_key=PINECONE_API_KEY)

In [25]:
from langchain_pinecone import PineconeVectorStore

index_name = "remote-lock"

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)


In [26]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
dense_vector = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
dense_retriever = dense_vector.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [28]:
dense_retriever.invoke("what is remote lock?")

[Document(id='7445c5f2-38bb-483b-a966-9a18b5e9d92d', metadata={'author': '', 'creationdate': '2025-08-27T12:43:18+05:00', 'creator': 'PyPDF', 'moddate': '2025-08-27T12:43:18+05:00', 'page': 36.0, 'page_label': '37', 'producer': 'Microsoft: Print To PDF', 'source': 'Data\\rdoc.pdf', 'title': 'Microsoft Word - remote lock data.docx', 'total_pages': 41.0}, page_content='Why RemoteLock? \nProperty managers of multiple (more than five) properties or units choose RemoteLock for \nthree key reasons: \n\uf0b7 Flexible scheduling capabilities. Create door groups, limited access for maintenance and \ncleaning crews and access to shared and common doors. \n\uf0b7 No unique hardware is required. RemoteLock is compatible with many leading smart \nlock brands. See all hardware. \n\uf0b7 Integrated with your property management or booking software. Easily generate and \ntrack access codes through your preferred dashboard. See all software partners. \nSoftware Features \nRemotely Control Locks & Manag

In [29]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [31]:
sparse_retriever=BM25Retriever.from_documents(text_chunks)
sparse_retriever.k=3 ##top- k documents to retriever

In [32]:
hybrid_retriever=EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weight=[0.7,0.3]
)

In [33]:
query = "what is remote lock?"
results = hybrid_retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}")


🔹 Document 1:
Why RemoteLock? 
Property managers of multiple (more than five) properties or units choose RemoteLock for 
three key reasons: 
 Flexible scheduling capabilities. Create door groups, limited access for maintenance and 
cleaning crews and access to shared and common doors. 
 No unique hardware is required. RemoteLock is compatible with many leading smart 
lock brands. See all hardware. 
 Integrated with your property management or booking software. Easily generate and 
track access codes through your preferred dashboard. See all software partners. 
Software Features 
Remotely Control Locks & Manage Users 
Control connected locks and manage users from your smartphone or laptop, from anywhere. 
Automate Guest Access

🔹 Document 2:
Integrations: Dozens of leading hardware and software providers (logos shown). See Software 
Partners and Hardware Partners pages. 
Customer quotes (examples): 
 “The fact that RemoteLock works with a variety of smart lock brands is a big advan

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [36]:
prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

In [37]:
### Create stuff Docuemnt Chain
document_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)

## create Full rAg chain
rag_chain=create_retrieval_chain(retriever=hybrid_retriever,combine_docs_chain=document_chain)
rag_chain


RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['PineconeVectorStore', 'OpenAIEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000013CB5791CC0>, search_kwargs={'k': 3}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000013CB5CDEAA0>, k=3)], weights=[0.5, 0.5]), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {input}\n')
          

In [39]:
response = rag_chain.invoke({"input": "How do i my account settings"})
print(response["answer"])

To access and modify your account settings, follow these steps:

1. Click the drop-down menu next to your account name located in the top-right corner of the interface.
2. Select **My Account** from the options presented.
3. In the **Profile Management** section under **My Account → Profile**, you can:
   - Edit your name, email, and username.
   - Change your password.
   - Add a phone number for Two-Factor Authentication (2FA).
4. After making any desired changes, be sure to click **Save** to confirm updates.
